In [1]:
%use dataframe
%use kandy
%use dataframe

In [2]:
import org.jetbrains.kotlinx.dataframe.DataFrame
import org.jetbrains.kotlinx.dataframe.api.*

class DataframeAnalyzer(private val dataframe: DataFrame<*>) {
    fun filter(filter: RowFilter<*>): DataframeAnalyzer {
        return DataframeAnalyzer(dataframe.filter(filter))
    }
    
    fun greaterThan(column: String, value: Double): DataframeAnalyzer {
        return filter { (it[column].toString().toDoubleOrNull() ?: return@filter false) > value }
    }
    
    fun lessThan(column: String, value: Double): DataframeAnalyzer {
        return filter { (it[column].toString().toDoubleOrNull() ?: return@filter false) < value }
    }
    
    fun doubleColumns(vararg columns: String): DataframeAnalyzer {
        return DataframeAnalyzer(dataframe.convert(*columns).with {
            it.toString().toDoubleOrNull()
        })
    }
    
    fun createPer90(column: String, ninetiesColumn: String = "Tempo de jogo - 90s"): DataframeAnalyzer {
        return DataframeAnalyzer(dataframe.insert(column + "/90") {
            (it[column].toString().toDoubleOrNull() ?: return@insert null) / (it[ninetiesColumn].toString().toDoubleOrNull() ?: error("ninetiesColumn is null"))
        }.after(column))
    }
    
    fun finish(): DataFrame<*> {
        return dataframe
    }
}

fun analyze(dataframe: DataFrame<*>): DataframeAnalyzer =
    DataframeAnalyzer(dataframe)

In [3]:
import org.jetbrains.kotlinx.kandy.ir.Plot

fun DataFrame<*>.footballPlot(
    x: String,
    y: String,
    min90s: Double = 0.0,
    per90: Boolean = false,
    ninetiesColumn: String = "Tempo de jogo - 90s"
): Plot {
    var analyzed = analyze(this)
        .doubleColumns(x, y)
        .greaterThan("90s", min90s)
    
    if (per90) {
        analyzed = analyzed.createPer90(x, ninetiesColumn).createPer90(y, ninetiesColumn)
    }
    
    return analyzed.finish().plot { 
        points {
            x(x + if (per90) "/90" else "")
            y(y + if (per90) "/90" else "")
            tooltips("Jogador", "Equipe")
        }
    }
}



In [27]:
class ComplexFootballPlot(var dataframe: DataFrame<*>) {
    private var x: AxisWrapper? = null
    private var y: AxisWrapper? = null
    private var tooltips: Array<out String> = emptyArray()
    
    fun x(column: String, block: AxisWrapper.() -> Unit) {
        val newX = AxisWrapper(column).apply(block)
        convertColumns(newX.column)
        if (newX.dividingColumn != null) {
            divide(newX.column, ninetiesColumn = newX.dividingColumn!!)
        }
        x = newX
    }
    
    fun y(column: String, block: AxisWrapper.() -> Unit) {
        val newY = AxisWrapper(column).apply(block)
        convertColumns(newY.column)
        if (newY.dividingColumn != null) {
            divide(newY.column, ninetiesColumn = newY.dividingColumn!!)
        }
        y = newY
    }
    
    fun convertColumns(vararg columns: String) {
        dataframe = dataframe.convert(*columns).with {
            it.toString().toDoubleOrNull()
        }
    }

    fun position(position: String, columnName: String = "Pos") {
        dataframe = dataframe.filter { it[columnName].toString().contains(position) }
    }

    fun strictPosition(position: String, columnName: String = "Pos") {
        dataframe = dataframe.filter { it[columnName] == position }
    }

    fun minimum(column: String, value: Double) {
        convertColumns(column)
        dataframe = dataframe.filter { (it[column].toString().toDoubleOrNull() ?: return@filter false) > value }
    }

    fun maximum(column: String, value: Double) {
        convertColumns(column)
        dataframe = dataframe.filter { (it[column].toString().toDoubleOrNull() ?: return@filter false) < value }
    }

    fun divide(vararg columns: String, divingColumn: String = "Tempo de jogo - 90s", suff) {
        convertColumns(*columns, ninetiesColumn)
        for (column in columns) {
            dataframe = dataframe.insert(column + "/90") {
                (it[column].toString().toDoubleOrNull() ?: return@insert null) / (it[ninetiesColumn].toString()
                    .toDoubleOrNull() ?: error("ninetiesColumn is null"))
            }.after(column)
        }
    }
    
    fun tooltips(vararg columns: String) {
        tooltips = columns
    }
    
    fun finish(): Plot {
        return dataframe.plot {
            points {
                x(this@ComplexFootballPlot.x?.column ?: error("x is null")) {
                    axis.name = this@ComplexFootballPlot.x!!.text
                }
                y(this@ComplexFootballPlot.y?.column ?: error("y is null")) {
                    axis.name = this@ComplexFootballPlot.y!!.text
                }
                tooltips(*tooltips)
            }
        }
    }
}

inline fun DataFrame<*>.complexPlot(block: ComplexFootballPlot.() -> Unit): Plot {
    val complexPlot = ComplexFootballPlot(this)
    complexPlot.block()
    return complexPlot.finish()
}

class AxisWrapper(
    val column: String
) {
    var text: String = column
    var dividingColumn: String? = null
    
    val actualColumn = 
}

In [28]:
val dataframe = DataFrame.readExcel("brasileirao.xlsx")

In [33]:
dataframe.footballPlot(
    x = "Progressão - PrgP",
    y = "Progressão - PrgC",
    min90s = 0.0,
    per90 = true
)

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="bhTdDQ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"Progressão - PrgC/90":[0.0,0.0,2.857142857142857,0.0,1.25,0.8695652173913044,0.7317073170731708,0.3333333333333333,10.0,0.0,2.857142857142857,1.0638297872340425,0.5263157894736842,0.0,0.0,0.625,0.3333333333333333,0.0,2.439024390243903,3.0,3.7037037037037033,0.2,0.0,4.0,2.4,2.7586206896551726,0.0,5.0,0.0,0.3846153846153846,0.35714285714285715,1.25,0.0,1.8181818181818183,0.0,0.0,1.3333333333333333,0.3448275862068966,0.0,0.22727272727272727,0.0,0.0,0.0,1.0,5.0,4.615384615384615,0.0,0.7142857142857143,4.2105263157894735,0.0,0.5,0.0,0.75,0.6666666666666666,2.857142857142857,0.3448275862068966,0.25,0.0,0.0,0.625,0.0,0.35714285714285715,2.727272727272727,0.45454545454545453,4.0,0.0,0.0,0.2,0.2173913043478261,3.6666666666666665,0.5263157894736842,null,0.0,2.608695652173913,0.0,8.0,0.8333333333333334,0.0,0.0,2.0,2.5,1.4814814814814814,0.5,1.6666666666666667,1.0,4.0,0.7692307692307692,3.888888888888889,1.6129032258064515,1.1764705882352942,2.6666666666666665,0.0,0.0,2.0,0.0,2.127659574468085,1.0714285714285714,1.6,0.4,1.4285714285714286,0.0,10.0,0.8,1.0,1.5384615384615383,2.0,1.0,0.0,2.0,0.0,0.0,0.0,0.3448275862068966,2.0,1.2244897959183672,3.6363636363636362,0.9677419354838709,1.6,null,1.7647058823529411,0.0,0.0,0.0,5.0,0.0,0.0,0.0,1.7391304347826089,1.1627906976744187,0.0,0.0,0.0,1.3333333333333333,4.615384615384615,0.0,1.4893617021276595,2.727272727272727,2.307692307692308,0.5555555555555556,9.0,0.0,0.5,1.6,0.0,1.5625,0.0,0.0,0.0,0.7692307692307692,1.4285714285714286,1.3513513513513513,5.0,5.0,1.0,1.0,0.0,1.5789473684210527,0.0,2.3333333333333335,0.0,0.0,0.0,1.3157894736842106,5.0,2.2727272727272725,16.666666666666668,0.0,2.608695652173913,0.0,0.45454545454545453,2.0,0.8333333333333334,5.833333333333334,2.2222222222222223,0.7894736842105263,0.0,0.0,0.0,15.0,4.4186046511627906,0.3846153846153846,0.0,1.6666666666666667,2.4489795918367343,2.5,0.0,null,0.6666666666666666,0.3333333333333333,1.0,1.4285714285714286,1.7857142857142858,1.75,0.0,0.0,1.081081081081081,3.3333333333333335,0.0,2.6666666666666665,10.0,2.0,0.0,0.6666666666666666,0.9090909090909091,0.0,2.0,0.0,0.5263157894736842,0.0,0.0,0.0,0.6,2.8125,0.0,0.9090909090909091,0.0,10.0,0.0,2.2222222222222223,1.25,6.666666666666667,0.0,0.0,0.0,2.2222222222222223,4.090909090909091,0.37037037037037035,0.0,1.1111111111111112,3.3333333333333335,1.3043478260869565,4.285714285714286,0.0,0.0,0.0,2.631578947368421,1.8,4.25,0.425531914893617,1.0,0.0,8.461538461538462,3.3333333333333335,0.8333333333333334,0.5,0.0,0.7407407407407407,0.0,0.0,2.5,0.0,0.9090909090909091,0.0,2.0,0.0,1.2820512820512822,2.0,0.0,0.3448275862068966,1.4285714285714286,4.0,2.75,4.4,0.47619047619047616,0.6,0.0,0.0,0.0,0.6896551724137931,1.8918918918918919,0.0,0.45454545454545453,1.8918918918918919,null,5.0,6.0,1.0714285714285714,0.0,0.0,0.6666666666666666,1.0,1.6666666666666667,0.0,0.7142857142857143,0.0,0.9090909090909091,6.153846153846153,0.0,0.5128205128205129,4.473684210526316,3.333333333333333,0.0,1.4285714285714286,0.0,3.3333333333333335,0.0,1.3333333333333333,0.0,4.545454545454545,0.0,0.0,1.4285714285714286,5.217391304347826,2.142857142857143,0.0,2.222222222222222,1.4,1.1764705882352942,0.0,1.1764705882352942,null,null,3.111111111111111,3.0769230769230766,3.3333333333333335,2.0,0.0,3.3333333333333335,1.4,0.0,0.0,1.6666666666666667,1.8,2.857142857142857,0.8620689655172414,2.5641025641025643,3.4782608695652177,2.368421052631579,0.0,3.0,1.3043478260869565,2.142857142857143,2.0,0.0,1.6666666666666667,0.0,2.0,0.0,0.0,1.0,1.0,1.25,1.1363636363636362,3.3333333333333335,0.7142857142857143,1.1764705882352942,0.9090909090909091,3

In [30]:
dataframe.complexPlot {
    x("Progressão - PrgP") {
        text = "Progressão com a bola"
        dividingColumn = "Tempo de jogo - 90s"
    }
    y("Progressão - PrgC") {
        text = "Progressão sem a bola"
        dividingColumn = "Tempo de jogo - 90s"
    }
    minimum("90s", 3.5)
    tooltips("Jogador", "Equipe")
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="7FCqeA"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"Progressão - PrgC":[3.0,5.0,2.0,0.0,10.0,12.0,1.0,0.0,20.0,12.0,1.0,0.0,2.0,3.0,1.0,0.0,2.0,1.0,1.0,2.0,4.0,10.0,8.0,4.0,8.0,6.0,8.0,0.0,0.0,5.0,0.0,7.0,9.0,2.0,0.0,0.0,5.0,4.0,9.0,5.0,0.0,3.0,0.0,19.0,12.0,10.0,7.0,4.0,0.0,2.0,3.0,4.0,15.0,9.0,17.0,2.0,0.0,5.0,11.0,22.0,3.0,7.0,7.0,2.0,17.0,9.0,7.0,14.0,7.0,9.0,5.0,10.0,9.0,0.0,15.0,5.0,15.0,9.0,21.0,0.0,4.0,1.0,0.0,7.0,5.0,0.0,0.0,10.0,12.0,3.0,6.0,3.0,5.0,13.0,3.0,1.0,1.0,3.0,1.0,2.0,11.0,0.0,0.0,13.0,3.0,10.0],
"Progressão - PrgP":[32.0,36.0,13.0,0.0,25.0,12.0,14.0,0.0,13.0,22.0,20.0,0.0,7.0,4.0,5.0,16.0,4.0,4.0,6.0,14.0,16.0,12.0,45.0,28.0,13.0,19.0,15.0,1.0,0.0,14.0,0.0,47.0,7.0,11.0,11.0,0.0,24.0,26.0,29.0,12.0,14.0,13.0,7.0,14.0,23.0,28.0,13.0,9.0,0.0,12.0,23.0,9.0,18.0,23.0,21.0,8.0,7.0,13.0,23.0,10.0,15.0,10.0,5.0,19.0,7.0,19.0,31.0,11.0,9.0,23.0,22.0,20.0,23.0,0.0,17.0,7.0,43.0,34.0,16.0,0.0,10.0,16.0,0.0,16.0,16.0,0.0,0.0,12.0,31.0,11.0,20.0,14.0,15.0,16.0,6.0,4.0,21.0,9.0,8.0,24.0,20.0,0.0,0.0,28.0,22.0,14.0]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Progressão com a bola",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Progressão sem a bola",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"Progressão - PrgP",
"y":"Progressão - PrgC"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"point",
"data":{
}
}]
};
 var plotContainer = document.getElementById("7FCqeA");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Progressão sem a bola 
 
 
 
 
 Progressão com a bola